# Crawl

> A module to help you crawl policing websites and ingest them for later use with your LLM

In [ ]:
#| default_exp crawl

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
import pandas as pd

We're just taking this code from the OpenAI tutorial. 

In [8]:

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# Function to get the hyperlinks from a URL
def get_hyperlinks(url):
    
    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:

            # Get the text from the URL using BeautifulSoup
            soup = BeautifulSoup(requests.get(url).text, "html.parser")

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")
            
            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)



We can now use that function to crawl the college of policing APP website.

In [4]:
# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

domain = "college.police.uk/app" # <- put your domain to be crawled
full_url = "https://www.college.police.uk/app" # <- put your domain to be crawled with https or http


crawl(full_url)

https://www.college.police.uk/app
HTTP Error 403: Forbidden


Clearly the College blocks IPs that harass their website, which is a little rude, but something I can work around.

In [7]:
# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

domain = "andreasthinks.me" # <- put your domain to be crawled
full_url = "https://andreasthinks.me/" # <- put your domain to be crawled with https or http


crawl(full_url)

https://andreasthinks.me/
https://andreasthinks.me/./posts/robbery_and_searches_in_space/robbery_and_searches_in_space.html
https://andreasthinks.me/../../recent_work.html
HTTP Error 400: Bad Request
https://andreasthinks.me/../../index.xml


/home/andreasthinksmint/python_env/police-risk-open-ai/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


HTTP Error 400: Bad Request
https://andreasthinks.me/../../about.html
HTTP Error 400: Bad Request
https://andreasthinks.me/../../index.html
HTTP Error 400: Bad Request
https://andreasthinks.me/./about.html
https://andreasthinks.me
https://andreasthinks.me/./posts/burglary_attendance/index.html
https://andreasthinks.me/./index.html
https://andreasthinks.me/./index.xml
https://andreasthinks.me/./posts/lapd_and_prophet/lapd_and_prophet.html
https://andreasthinks.me/./recent_work.html
https://andreasthinks.me/./posts/migrated_to_quarto/index.html
https://andreasthinks.me/./posts/lockdown_effect/index.html
https://andreasthinks.me/./posts/FastAI_Twitter_Sentiment_Analysis_Tutorial/FastAI_Twitter_Sentiment_Analysis_Tutorial.html


So interestingly this is soemthing the college has set up to block...how interesting.

Rather than requesting the URL directly, we'll inject in a header that looks like the Firefox browser.

```

    request = urllib.request.Request(url, headers={'User-Agent':'Mozilla/5.0'})
    
    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(request) as response:
```
I also add in a max length of the URL, because the College has some real weird stuff that is breaking my code.

```

        # Save text from the url to a <url>.txt file
        if len(url) < 500:
            with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:

```


That *should* let us scrape without being kicked off, we hope.

In [16]:
#| export


# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    request = urllib.request.Request(url, headers={'User-Agent':'Mozilla/5.0'})
    
    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(request) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))

def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        if len(url) < 500:
            with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w", encoding="UTF-8") as f:

                # Get the text from the URL using BeautifulSoup
                soup = BeautifulSoup(requests.get(url).text, "html.parser")

                # Get the text but remove the tags
                text = soup.get_text()

                # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                if ("You need to enable JavaScript to run this app." in text):
                    print("Unable to parse page " + url + " due to JavaScript being required")
                
                # Otherwise, write the text to the file in the text directory
                f.write(text)

            # Get the hyperlinks from the URL and add them to the queue
            for link in get_domain_hyperlinks(local_domain, url):
                if link not in seen:
                    queue.append(link)
                    seen.add(link)



In [17]:
# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

domain = "college.police.uk/app" # <- put your domain to be crawled
full_url = "https://www.college.police.uk/app" # <- put your domain to be crawled with https or http


crawl(full_url)

https://www.college.police.uk/app
https://www.college.police.uk/about
https://www.college.police.uk/about/concordats
https://www.college.police.uk/about/publication-scheme
https://www.college.police.uk/about/data-protection-foi/individual-rights
https://www.college.police.uk/cdn-cgi/l/email-protection#11757065703f61637e65747265787e7f51727e7d7d7476743f617e7d7872743f647a
https://www.college.police.uk/cdn-cgi/l/email-protection#9df9fce9fcb3edeff2e9f8fee9f4f2f3ddfef2f1f1f8faf8b3edf2f1f4fef8b3e8f6
https://www.college.police.uk/diversity-and-inclusion
https://www.college.police.uk/article/inclusion-inside-out-strategic-approach
https://www.college.police.uk/article/neurodiversity-training-better-support-public
https://www.college.police.uk/tags/going-equipped
https://www.college.police.uk/article/socialisation-student-officers
https://www.college.police.uk/cdn-cgi/l/email-protection#27185452454d4244531a7448444e464b4e5446534e48490748410754535243424953074841414e44425554014548435e1a6e0215175446

/tmp/ipykernel_90217/114368700.py:107: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(requests.get(url).text, "html.parser")


https://www.college.police.uk/app/stop-and-search/legal/legal-basis
https://www.college.police.uk/cdn-cgi/l/email-protection#f3cc80869199969087cebf9694929fd39192809a80d5919c978acebad6c1c3809284d6c1c3879b9a80d6c1c39c9dd6c1c3879b96d6c1c3b0bca3d6c1c3d6c1c3849691809a8796d6c1c3929d97d6c1c3879b9c86949b87d6c1c38a9c86d6c1c39e9a949b87d6c1c39196d6c1c39a9d8796819680879697d6c3b7d6c3b2bf9694929fd39192809a80d6c3b7d6c3b29b87878380c9dcdc848484dd909c9f9f969496dd839c9f9a9096dd8698dc928383dc80879c83de929d97de80969281909bdc9f9694929fdc9f9694929fde9192809a80
https://www.college.police.uk/app/stop-and-search/quick-reference-guides/quick-reference-guide-professional-interaction
https://www.college.police.uk/print/pdf/node/2635
https://www.college.police.uk/cdn-cgi/l/email-protection#8cb3fff9eee6e9eff8b1ddf9e5efe7acfee9eae9fee9e2efe9acebf9e5e8e9b6acfcfee3eae9ffffe5e3e2ede0ace5e2f8e9feedeff8e5e3e2aaeee3e8f5b1c5a9bebcffedfba9bebcf8e4e5ffa9bebce3e2a9bebcf8e4e9a9bebccfc3dca9bebca9bebcfbe9eeffe5f8e9a9bebcede2e8a9b

In [13]:
test_srt = 'text/www.college.police.uk/www.college.police.uk_cdn-cgi_l_email-protection#1a25696f78707f796e27496f7379737e7f3a6a687f6c7f746e7375743a7b747e3a687369713a777b747b7d7f777f746e3a7c75683a6a7f686a7f6e687b6e7568693a757c3a797273767e3a697f626f7b763a7f626a7675736e7b6e7375743a7b747e3a73747e7f797f746e3a73777b7d7f693a757c3a797273767e687f743a3253535559333c78757e6327533f282a697b6d3f282a6e7273693f282a75743f282a6e727f3f282a59554a3f282a3f282a6d7f7869736e7f3f282a7b747e3f282a6e72756f7d726e3f282a63756f3f282a77737d726e3f282a787f3f282a73746e7f687f696e7f7e3f2a5e3f2a5b496f7379737e7f3a6a687f6c7f746e7375743a7b747e3a687369713a777b747b7d7f777f746e3a7c75683a6a7f686a7f6e687b6e7568693a757c3a797273767e3a697f626f7b763a7f626a7675736e7b6e7375743a7b747e3a73747e7f797f746e3a73777b7d7f693a757c3a797273767e687f743a3253535559333f2a5e3f2a5b726e6e6a692035356d6d6d34797576767f7d7f346a757673797f346f71357b6a6a357e7f6e7f746e737574377b747e37796f696e757e6335696f7379737e7f376a687f6c7f746e737574377b747e376873697137777b747b7d7f777f746e376a7f686a7f6e687b6e75686937797273767e37697f626f7b76377f626a7675736e7b6e737574377b747e3773747e7f797f746e3773777b7d7f6937797273767e687f743773737579.txt'
len(test_srt)

1146

Perfect! With the website nudged into think I'm a normal user, I've now scrapped a whole bunch of the APP.  Let's now take all our documens, clean up the junk (remember those awkward new-line characters), and add them to a single CSV file we're use as our "brain".

That said...I've not just scrapped the APP, I seem to have got *everything* on the public College website.  Which is good on one hand, but also going to be a little messy. On a relevant sidenote, why isn't the APP available through an API as a digital product?

Now lets ingest all our data into a single clean CSV.  This is more code from the open API tutorial, cleaned up a little for my use.

In [ ]:
#| export

def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

def clean_scrapped_data(scrape_directory, output_file='processed/scraped.csv'):
    """ takes a folder containing all the file from your scrapped data, cleans it all, saves as a CSV and returns the dataframe

    if given None as output_file, dataframe returned but not saved"""

    # Create a list to store the text files
    texts=[]

    # Get all the text files in the text directory
    for file in os.listdir(scrape_directory + "/"):

        # Open the file and read the text
        with open("text/" + domain + "/" + file, "r", encoding="UTF-8") as f:
            text = f.read()

            # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
            texts.append((file[11:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

    # Create a dataframe from the list of texts
    df = pd.DataFrame(texts, columns = ['fname', 'text'])

    # Set the text column to be the raw text with the newlines removed
    df['text'] = df.fname + ". " + remove_newlines(df.text)
    if output_file is not None:
        df.to_csv(output_file)
    return df

def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie